In [ ]:
# Import the libraries
import os, json, ast
import openai
# from tenacity import retry, wait_random_exponential, stop_after_attempt
from google.colab import userdata


In [ ]:
# # If you're using the default OpenAI API key, uncomment the following lines:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### 1. Initialize Conversation with System Prompt + Few-shot

In [ ]:
def initialize_conversation():
    """
    Returns a rich initial prompt including:
    - A system message defining assistant personality and constraints
    - Multiple few-shot examples that demonstrate reasoning
    - Sets context for consistent user interaction
    """

    system = {
        "role": "system",
        "content": (
            "You are DubaiLocalExplorer, an intelligent assistant that helps tourists explore Dubai. "
            "You engage in natural conversation to understand user preferences, confirm intent, and suggest "
            "personalized day-trip itineraries. Use friendly, concise language. "
            "\n\nYour goal is to:\n"
            "- Help users clarify their interests, budget, group type, and duration\n"
            "- Ask clarifying questions if data is missing\n"
            "- Suggest experiences that match those constraints\n"
            "- Think step-by-step and summarize total cost & time\n\n"
            "Always tailor suggestions based on their profile. Keep the tone warm and adaptive."
        )
    }

    few_shot = [
        {
            "role": "user",
            "content": "Hi! I'm in Dubai for 1 day. I'd love to explore cultural sites and try local food."
        },
        {
            "role": "assistant",
            "content": (
                "That sounds like a wonderful day! Here's a quick plan:\n"
                "1. Start at Al Fahidi Historical District to explore Dubai Museum.\n"
                "2. Ride an abra across the Creek to the Gold and Spice Souks.\n"
                "3. Try Emirati cuisine at Al Seef or Arabian Tea House.\n\n"
                "Would you prefer walking between places, or prefer transport arranged?"
            )
        },
        {
            "role": "user",
            "content": "I'm traveling with my wife, and we have around 700 AED to spend."
        },
        {
            "role": "assistant",
            "content": (
                "Thanks for sharing! That gives us good flexibility.\n"
                "I'll make sure all experiences are within budget for a couple. "
                "Would you like to include modern attractions like Dubai Frame or Museum of the Future?"
            )
        },
        {
            "role": "user",
            "content": "We are a group of friends, want nightlife and beaches, budget is tight though."
        },
        {
            "role": "assistant",
            "content": (
                "Got it — you're a group looking for beach and nightlife fun, but mindful of costs.\n"
                "Let me find beach clubs or night spots with free or low entry fees, and suggest group-friendly spots. "
                "Can you tell me how many days you’re staying?"
            )
        },
        {
            "role": "user",
            "content": "Just one full day. Don’t want to waste time in malls."
        },
        {
            "role": "assistant",
            "content": (
                "Understood — one focused day without shopping distractions.\n"
                "How about this:\n"
                "- Morning: Relax at Kite Beach (free entry, lots of cafés)\n"
                "- Afternoon: Explore JBR beachfront and walk The Beach boardwalk\n"
                "- Evening: Rooftop dinner at a budget-friendly lounge near Dubai Marina\n\n"
                "Want me to include transport suggestions?"
            )
        }
    ]

    return [system] + few_shot


In [ ]:
def get_chat_model_completions(conversation, model="gpt-4-turbo", temperature=0.4, print_response=True):
    """
    Submits a message list (conversation) to the OpenAI ChatCompletion API and prints the assistant's response.
    
    Args:
        conversation (list): List of dicts like [{"role": "user", "content": "..."}, ...]
        model (str): LLM model to use (default: "gpt-4-turbo")
        temperature (float): Sampling temperature
        print_response (bool): Whether to print response to console

    Returns:
        str: Assistant's message content
    """

    # Validate input
    if not isinstance(conversation, list) or not all("role" in m and "content" in m for m in conversation):
        raise ValueError("Conversation must be a list of dicts with 'role' and 'content' keys")

    try:
        # Make API call
        response = openai.ChatCompletion.create(
            model=model,
            messages=conversation,
            temperature=temperature
        )

        message = response.choices[0].message.content.strip()

        if print_response:
            print("\n Assistant Response:\n")
            print(message)
        
        return message

    except openai.error.OpenAIError as e:
        print(f" OpenAI API error: {e}")
        return "An error occurred while contacting the language model."


In [ ]:
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What are some fun things to do in Dubai?"}
]
get_chat_model_completions(conversation)


In [ ]:
conversation = [
    {"role": "system", "content": "You are a travel assistant."},
    {"role": "user", "content": "I'm a couple visiting Dubai for 2 days. Love culture and food."},
    {"role": "user", "content": "We want to keep it under 600 AED."}
]
get_chat_model_completions(conversation)


In [ ]:
conversation = [
    {"role": "system", "content": "You extract user travel preferences into JSON format."},
    {"role": "user", "content": "I'm coming with kids, want beach and outdoor stuff. Budget is around 700 AED."},
    {"role": "user", "content": "Staying for one day only."},
    {"role": "user", "content": (
        "Please extract my travel preferences as structured JSON with keys: "
        "'interests', 'duration_days', 'budget_aed', 'group_type'."
    )}
]
get_chat_model_completions(conversation)


In [ ]:
debug_user_input = 'my max budget is 1000 AED'

In [ ]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

In [ ]:
def moderation_check(user_input):
    """
    Uses OpenAI's Moderation API to check if input violates content policy.
    Returns a dict with moderation status and flagged categories.
    """
    try:
        response = openai.Moderation.create(input=user_input)
        results = response["results"][0]

        return {
            "flagged": results["flagged"],
            "categories": results["categories"]
        }

    except openai.error.OpenAIError as e:
        print(f"Moderation API error: {e}")
        return {
            "flagged": False,
            "categories": {},
            "error": str(e)
        }


In [ ]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

In [ ]:
print(moderation_check("I want to kill Ravan."))
print(moderation_check("Need a Dubai Desert Safari Package"))

**Let's now check moderation on the assistant's response debug_response_assistant.**

In [ ]:
moderation_check(debug_response_assistant)

### 2. Intent & Profile Confirmation Layer

In [ ]:
intent_extraction_prompt = [
    {"role": "system", "content": (
        "You are a helpful assistant that extracts travel intent as structured JSON. "
        "Your goal is to return a user profile with four keys: 'interests', 'duration_days', 'budget_aed', 'group_type'. "
        "If any values are missing, return them under a second key called 'missing_info'. "
        "Always think step-by-step before creating the JSON."
    )},
    
    {"role": "user", "content": "I want to visit some cultural places and markets in Dubai. I have about 500 AED to spend."},
    {"role": "assistant", "content": (
        "Step-by-step reasoning:\n"
        "- The user mentions cultural places and markets → interests = ['culture', 'shopping']\n"
        "- They mention a budget of 500 AED → budget_aed = 500\n"
        "- Duration is unspecified → missing_info = ['duration_days']\n"
        "- Group type not mentioned → missing_info += ['group_type']\n"
        "{\n"
        "  'interests': ['culture', 'shopping'],\n"
        "  'budget_aed': 500,\n"
        "  'missing_info': ['duration_days', 'group_type']\n"
        "}"
    )},

    {"role": "user", "content": "We are a couple traveling for 2 days, looking for adventure and beach activities."},
    {"role": "assistant", "content": (
        "Step-by-step reasoning:\n"
        "- Interests: adventure and beach → interests = ['adventure', 'beach']\n"
        "- Duration: 2 days\n"
        "- Group type: couple\n"
        "- Budget not mentioned → missing_info = ['budget_aed']\n"
        "{\n"
        "  'interests': ['adventure', 'beach'],\n"
        "  'duration_days': 2,\n"
        "  'group_type': 'couple',\n"
        "  'missing_info': ['budget_aed']\n"
        "}"
    )},
]


In [ ]:
def intent_confirmation_layer(conversation):
    """
    Extracts the user profile (interests, duration, budget, group_type) 
    using few-shot prompting and returns missing fields if needed.
    """

    # Add few-shot prompt history
    prompt = intent_extraction_prompt + conversation + [{
        "role": "user",
        "content": (
            "Please extract my travel preferences as structured JSON. "
            "Use the format shown earlier with step-by-step reasoning."
        )
    }]

    # LLM Call
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=prompt,
        temperature=0.4
    )

    result_text = response.choices[0].message.content.strip()

    # Attempt to evaluate the returned JSON safely
    import json
    try:
        # Strip reasoning if needed
        json_part = result_text[result_text.find("{"):]
        profile = json.loads(json_part)
    except Exception as e:
        profile = {"error": "Failed to parse LLM output.", "raw": result_text}

    return profile

In [ ]:
def dictionary_present(user_request):
    """
    Sends the user request to the OpenAI Completion API with few-shot examples,
    expecting a valid dictionary in response.

    Returns:
        dict if dictionary is valid, otherwise None
    """

    # Few-shot examples to guide the model
    few_shot_prompt = """
            You are an assistant that extracts structured travel intent in Python dictionary format.
            
            Example 1:
            User: I want to explore historical sites and shop for souvenirs. I'm coming with my family for 3 days and want to spend about 800 AED.
            Response:
            {
              "interests": ["culture", "shopping"],
              "duration_days": 3,
              "budget_aed": 800,
              "group_type": "family"
            }
            
            Example 2:
            User: We are a couple interested in beaches and relaxing spots. We’ll be here just one day and can spend around 500 AED.
            Response:
            {
              "interests": ["beach", "relaxation"],
              "duration_days": 1,
              "budget_aed": 500,
              "group_type": "couple"
            }
            
            Example 3:
            User: {user_request}
            Response:
            """.strip().format(user_request=user_request)

    # Generate output using Completion endpoint (not Chat)
    try:
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt=few_shot_prompt,
            max_tokens=150,
            temperature=0.3,
            stop=["\n\n"]
        )

        response_text = response.choices[0].text.strip()

        # Attempt to parse the dictionary from model output
        parsed_dict = ast.literal_eval(response_text)
        if isinstance(parsed_dict, dict):
            return parsed_dict
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None


In [ ]:
user_request = "I'm visiting with friends, want to party and enjoy nightlife. Staying for 2 days with a budget of 600 AED."
profile_dict = dictionary_present(user_request)

if profile_dict:
    print("Parsed profile:", profile_dict)
else:
    print("Could not extract dictionary.")


### iterate_response() - Helper Function:
We've created a small helper test function to ensure the model's response is consistent.
Uncomment the code blocks and run the function `iterate_response(response)` to check if the response of the `intent_confirmation_layer`is consistent.}

In [ ]:
def iterate_llm_response(funct, debug_response, num = 10):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

# Example usage: test the consistency of responses from 'intent_confirmation_layer'
# iterate_llm_response(get_chat_completions, messages)

In [ ]:
response_as_dict_for_intent_confirmed

In [ ]:
# Check for LLM function's consistency
iterate_llm_response(dictionary_present, response_as_dict_for_intent_confirmed)

In [ ]:
def product_mapping(user_profile, products=dubai_products):
    """
    Maps user profile to matching Dubai products.

    Args:
        user_profile (dict): Extracted user info with keys:
            'interests' (list[str]), 'budget_aed' (int),
            'duration_days' (int), 'group_type' (str)

        products (list[dict]): Catalog of products/experiences

    Returns:
        list[dict]: Matching products sorted by relevance
    """

    matches = []

    user_interests = set(i.lower() for i in user_profile.get("interests", []))
    budget = user_profile.get("budget_aed", 0)
    group = user_profile.get("group_type", "").lower()

    for product in products:
        tags = set(t.lower() for t in product["tags"])
        if user_interests.intersection(tags):
            # Check budget fit
            if product["min_budget"] <= budget <= product["max_budget"]:
                # Check group suitability
                if group in [g.lower() for g in product["suitable_for"]]:
                    matches.append(product)

    # Optional: Sort by how many tags match (descending)
    matches.sort(key=lambda p: len(user_interests.intersection(set(t.lower() for t in p["tags"]))), reverse=True)

    return matches


In [ ]:
def extract_information(raw_text):
    """
    Uses LLM to extract structured info (interests, budget, duration, group_type)
    from raw user input or assistant output.

    Args:
        raw_text (str): Raw text input from user or assistant

    Returns:
        dict: Extracted info dictionary or None if extraction fails
    """

    prompt = f"""
                Extract the following information from the user's input as a Python dictionary with keys:
                - interests (list of strings)
                - duration_days (integer)
                - budget_aed (integer)
                - group_type (string, e.g., family, couple, friends, solo)
                
                User Input: \"\"\"{raw_text}\"\"\"
                
                Extracted Info:
                """

    try:
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt=prompt,
            max_tokens=150,
            temperature=0.2,
            stop=["\n\n"]
        )

        response_text = response.choices[0].text.strip()
        parsed_dict = ast.literal_eval(response_text)
        if isinstance(parsed_dict, dict):
            return parsed_dict
        else:
            return None

    except Exception as e:
        print(f"Error extracting information: {e}")
        return None


In [ ]:
# Raw user input
user_input = "I'm a couple visiting Dubai for 2 days. We love culture and food with a budget of 600 AED."

# Step 1: Extract info
profile = extract_information(user_input)
print("Extracted profile:", profile)

# Step 2: Map products
if profile:
    recommended = product_mapping(profile)
    print("\nRecommended Experiences:")
    for p in recommended:
        print(f"- {p['name']}: {p['description']}")
else:
    print("Could not extract user profile.")


In [ ]:
def match_experiences_to_profile(user_profile, experiences=dubai_experiences):
    """
    Match Dubai experiences to user profile with scoring and detailed reasoning.

    Args:
        user_profile (dict): Dictionary with keys:
            - interests (list[str])
            - budget_aed (int)
            - duration_days (int)
            - group_type (str)

        experiences (list[dict]): List of Dubai experiences.

    Returns:
        list of dict: Sorted recommended experiences with scores and reasons.
    """

    recommendations = []

    user_interests = set(i.lower() for i in user_profile.get("interests", []))
    budget = user_profile.get("budget_aed", 0)
    duration_days = user_profile.get("duration_days", 0)
    group = user_profile.get("group_type", "").lower()

    # Convert duration_days to hours roughly (8 hours/day for activities)
    user_available_hours = duration_days * 8

    for exp in experiences:
        exp_tags = set(t.lower() for t in exp["tags"])
        exp_min_budget = exp["min_budget"]
        exp_max_budget = exp["max_budget"]
        exp_duration = exp["duration_hours"]
        exp_groups = [g.lower() for g in exp["suitable_for"]]

        # Score calculation parts:
        score = 0
        reasons = []

        # Interest overlap score (max 40 points)
        interest_overlap = len(user_interests.intersection(exp_tags))
        interest_score = interest_overlap * 10  # each matching interest = 10 points
        if interest_score > 40:
            interest_score = 40
        score += interest_score
        reasons.append(f"Interest overlap: {interest_overlap} matches, +{interest_score} pts")

        # Budget fit (max 30 points)
        if exp_min_budget <= budget <= exp_max_budget:
            budget_score = 30
            reasons.append(f"Budget {budget} AED fits in range [{exp_min_budget}-{exp_max_budget}], +{budget_score} pts")
        else:
            # Penalize if out of budget range
            budget_score = 0
            reasons.append(f"Budget {budget} AED out of range [{exp_min_budget}-{exp_max_budget}], +0 pts")
        score += budget_score

        # Duration fit (max 20 points)
        if exp_duration <= user_available_hours:
            duration_score = 20
            reasons.append(f"Duration {exp_duration}h fits in available {user_available_hours}h, +{duration_score} pts")
        else:
            duration_score = 0
            reasons.append(f"Duration {exp_duration}h exceeds available {user_available_hours}h, +0 pts")
        score += duration_score

        # Group suitability (max 10 points)
        if group in exp_groups:
            group_score = 10
            reasons.append(f"Suitable for group '{group}', +{group_score} pts")
        else:
            group_score = 0
            reasons.append(f"Not suitable for group '{group}', +0 pts")
        score += group_score

        recommendations.append({
            "experience": exp["name"],
            "description": exp["description"],
            "score": score,
            "reasons": reasons
        })

    # Sort by descending score
    recommendations.sort(key=lambda x: x["score"], reverse=True)

    return recommendations


In [ ]:
user_profile = {
    "interests": ["culture", "shopping", "beach"],
    "budget_aed": 600,
    "duration_days": 2,
    "group_type": "family"
}

recommendations = match_experiences_to_profile(user_profile)

print("Recommendations for user:")
for rec in recommendations:
    print(f"\nExperience: {rec['experience']} (Score: {rec['score']})")
    print("Description:", rec['description'])
    print("Reasons:")
    for reason in rec['reasons']:
        print("-", reason)


In [ ]:
def product_recommendation_layer(recommendations, top_n=3):
    """
    Prepare a formatted product recommendation message from experience matches.

    Args:
        recommendations (list of dict): Output from match_experiences_to_profile
        top_n (int): Number of top recommendations to include

    Returns:
        str: Formatted user-facing recommendation message
    """

    if not recommendations:
        return "Sorry, I couldn't find any experiences matching your preferences. Could you please provide more details or adjust your preferences?"

    message_lines = ["Based on your preferences, here are some experiences I recommend in Dubai:\n"]

    for rec in recommendations[:top_n]:
        message_lines.append(f"**{rec['experience']}** (Score: {rec['score']})")
        message_lines.append(f"{rec['description']}")
        # Optionally include reasons, or summarize them
        reasons_summary = "; ".join(rec['reasons'])
        message_lines.append(f"Why? {reasons_summary}\n")

    message_lines.append("Would you like more information on any of these, or help with booking?")

    return "\n".join(message_lines)


In [ ]:
def initialize_conv_reco(user_profile):
    """
    Initialize conversation with product recommendations based on user profile.

    Args:
        user_profile (dict): Extracted user preferences dictionary

    Returns:
        str: Assistant message with product recommendations
    """

    # Step 1: Match experiences
    recommendations = match_experiences_to_profile(user_profile)

    # Step 2: Generate recommendation message
    recommendation_message = product_recommendation_layer(recommendations)

    # You can add additional personalization or conversation starter text here

    return recommendation_message


In [ ]:
user_profile = {
    "interests": ["culture", "shopping", "beach"],
    "budget_aed": 600,
    "duration_days": 2,
    "group_type": "family"
}

assistant_message = initialize_conv_reco(user_profile)
print(assistant_message)
